In [13]:
!python OOD_Generate_Mahalanobis.py --dataset cifar10 --outf output/ --net_type resnet --gpu 0

Namespace(batch_size=200, dataset='cifar10', dataroot='./data', outf='output/', num_classes=10, net_type='resnet', gpu=0)
load model: resnet
load target data:  cifar10
100%|██████████████████████| 170498071/170498071 [00:01<00:00, 100075350.48it/s]
Extracting ./data/cifar10-data/cifar-10-python.tar.gz to ./data/cifar10-data
Files already downloaded and verified
get sample mean and covariance
/home/zjiaming/deep_Mahalanobis_detector/lib_generation.py:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)

 Training Accuracy:(99.75%)

get Mahalanobis scores
Noise: 0.0
/home/zjiaming/deep_Mahalanobis_detector/lib_generation.py:179: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  ../torch/csrc/utils/python

In [19]:
!python OOD_Regression_Mahalanobis.py --net_type resnet

Namespace(net_type='resnet')
In-distribution:  cifar10
Out-of-distribution:  svhn
(36032, 5) (36032,)
in_distribution: cifar10==========
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 91.53  98.40  93.64  96.46  99.37
Input noise: Mahalanobis_0.0



In [52]:
import torch
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
         
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs


In [6]:
from torch.utils.data import Dataset, DataLoader

class ConfidenceDataset(Dataset):
    def __init__(self, confidence, labels):
        self.labels = labels
        self.confidence = confidence

    def __len__(self):
        return len(self.confidence)

    def __getitem__(self, idx):
        label = self.labels[idx]
        conf = self.confidence[idx]
        sample = {"Confidence": conf, "Class": label}
        return sample

In [13]:
import lib_regression
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression


epochs = 5000
input_dim = 5 # Two inputs x1 and x2 
output_dim = 1 # Single binary output 
learning_rate = 0.000001

criterion = torch.nn.BCELoss()
# criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

score = 'Mahalanobis_0.0'
dataset = 'cifar10'
out = 'svhn'
outf = 'output/resnet_cifar10/'
total_X, total_Y = lib_regression.load_characteristics(score, dataset, out, outf)
X_val, Y_val, X_test, Y_test = lib_regression.block_split(total_X, total_Y, out)
X_train = torch.tensor(np.concatenate((X_val[:500], X_val[1000:1500])), dtype=torch.float)

Y_train = torch.tensor(np.concatenate((Y_val[:500], Y_val[1000:1500])), dtype=torch.float)
train_dataset = ConfidenceDataset(X_train, Y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,
                                                shuffle=True, num_workers=1)
X_val_for_test = torch.tensor(np.concatenate((X_val[500:1000], X_val[1500:])), dtype=torch.float)
Y_val_for_test = torch.tensor(np.concatenate((Y_val[500:1000], Y_val[1500:])), dtype=torch.float)


lr = LogisticRegression(random_state=0).fit(X_train, Y_train)
y_pred = lr.predict_proba(X_train)[:, 1]
y_pred.round()


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [51]:
print(lr.intercept_)
lr.coef_

[-11.96437576]


array([[ 0.03915799,  0.02396634, -0.09159853, -0.00617044, -0.00793493]])

In [34]:
torch.sigmoid(torch.tensor([-100]))

tensor([0.])

In [41]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.1321, -0.1382, -0.2071, -0.3147, -0.3481]], requires_grad=True)


In [61]:
model = LogisticRegression(5, 1)
std_weight = torch.tensor([0.03915799,  0.02396634, -0.09159853, -0.00617044, -0.00793493], requires_grad=True)
std_bias = torch.tensor([-11.964], requires_grad=True)
with torch.no_grad():  
    print(model.linear.weight)
    print(model.linear.bias)
    model.linear.weight[0] = std_weight
    model.linear.bias[0] = std_bias
    print(model.linear.bias)
    print(model.linear.weight[0])
    # model.weight = torch.nn.Parameter(torch.ones_like(model.weight))


Parameter containing:
tensor([[ 0.0024,  0.1673,  0.3857, -0.1583, -0.0129]], requires_grad=True)
Parameter containing:
tensor([0.1928], requires_grad=True)
Parameter containing:
tensor([-11.9640], requires_grad=True)
tensor([ 0.0392,  0.0240, -0.0916, -0.0062, -0.0079], requires_grad=True)


In [48]:
X_train

tensor([[ -24.1423,  -35.2412,  -77.2508, -150.7342, -535.7014],
        [  -6.9233,  -29.1268,  -66.2837, -133.0634, -290.9829],
        [  -5.4603,  -25.4157,  -57.1742, -132.8230, -447.3622],
        ...,
        [  -9.5535,  -53.6929, -191.0907, -182.7891, -742.5851],
        [  -8.3014,  -56.0449, -200.5829, -194.7397, -725.3740],
        [  -5.5292,  -40.1784, -150.4286, -355.8837, -886.5150]])

In [54]:
x = -24*0.0392-35*0.024+0.0916*77.2508+150.7432*0.0062+0.0079*535.7014 - 11.964
1/(1 + np.exp(-x))


0.18213072423175303

In [62]:
model(X_train)

tensor([[1.8286e-01],
        [2.3392e-02],
        [3.9906e-02],
        [4.3836e-01],
        [6.1568e-02],
        [1.2224e-02],
        [2.9531e-01],
        [6.2023e-03],
        [1.6511e-03],
        [8.7593e-04],
        [3.1930e-03],
        [5.6098e-03],
        [1.4373e-02],
        [7.6628e-03],
        [2.5856e-03],
        [4.7609e-01],
        [4.1080e-02],
        [2.2722e-02],
        [9.2430e-04],
        [5.0291e-03],
        [6.7742e-02],
        [8.8321e-03],
        [1.3789e-01],
        [3.5793e-03],
        [3.5498e-02],
        [9.1221e-01],
        [6.3582e-02],
        [4.4090e-02],
        [6.7701e-03],
        [1.4116e-02],
        [6.9305e-03],
        [1.2394e-02],
        [2.1253e-02],
        [2.0491e-02],
        [4.4566e-04],
        [2.8461e-01],
        [8.9467e-03],
        [1.6125e-02],
        [4.7083e-02],
        [2.7872e-02],
        [5.3929e-02],
        [3.0015e-02],
        [2.3930e-02],
        [9.0503e-04],
        [5.6254e-03],
        [1

In [22]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.2443,  0.0947, -0.0014, -0.0274,  0.5002]], requires_grad=True)
Parameter containing:
tensor([0.3317], requires_grad=True)


In [46]:
from torch.autograd import Variable

import torch
# class LogisticRegression(torch.nn.Module):
#      def __init__(self, input_dim, output_dim):
#          super(LogisticRegression, self).__init__()
#          self.linear = torch.nn.Linear(input_dim, output_dim)
         
#      def forward(self, x):
#          outputs = torch.sigmoid(self.linear(x))
#          return outputs
# model = LogisticRegression(5, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.0001)

# Y_train = Variable(Y_train)
for epoch in tqdm(range(int(epochs))):
    for data_batch in train_loader:
        data = Variable(data_batch['Confidence'], requires_grad=True)
        labels = data_batch['Class']
        # print(data)
        optimizer.zero_grad()
        outputs = model(data)
        outputs = torch.squeeze(outputs)
        # outputs = outputs.view(outputs.size(0))
        # print(outputs.shape)
        loss = criterion(outputs, labels) 
        # print(loss)
        # break
        # loss = Variable(loss, requires_grad=True)
        # loss.backward()
        
        # optimizer.step()

    if epoch % 100 == 0:
        with torch.no_grad():
            total_test = 0
            correct_test = 0
            output_test = torch.squeeze(model(X_val_for_test))
            loss_test = criterion(output_test, Y_val_for_test)

            predicted_test = output_test.round().detach().numpy()
            total_test += Y_val_for_test.size(0)

            correct_test += np.sum(predicted_test == Y_val_for_test.detach().numpy())

            accuracy_test = 100 * correct_test/total_test

            print(f"Test - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")

            total = 0
            correct = 0
            total += Y_train.size(0)
            
            correct += np.sum(torch.squeeze(model(X_train)).round().detach().numpy() == Y_train.detach().numpy())
            accuracy = 100 * correct/total
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")

  0%|          | 0/5000 [00:00<?, ?it/s]

Test - Loss: 4.412589073181152. Accuracy: 50.0
Train -  Loss: 3.9846997261047363. Accuracy: 50.0



  2%|▏         | 100/5000 [00:06<05:43, 14.25it/s]

Test - Loss: 4.412589073181152. Accuracy: 50.0
Train -  Loss: 3.119951009750366. Accuracy: 50.0



  3%|▎         | 146/5000 [00:10<05:34, 14.49it/s]


KeyboardInterrupt: 

In [1]:
import models
from torchvision import transforms
import data_loader
import lib_generation
import torch
from torch.autograd import Variable
import numpy as np


model = models.ResNet34(num_c=10)
model.load_state_dict(torch.load("pre_trained/resnet_cifar10.pth", map_location = "cuda:" + str(0)))
in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

train_loader, test_loader = data_loader.getTargetDataSet('cifar10', 16, in_transform, './data')

model.cuda()
# set information about feature extaction
model.eval()
temp_x = torch.rand(2,3,32,32).cuda()
temp_x = Variable(temp_x)
temp_list = model.feature_list(temp_x)[1]
num_output = len(temp_list)
feature_list = np.empty(num_output)
count = 0
for out in temp_list:
    feature_list[count] = out.size(1)
    count += 1

sample_mean, precision = lib_generation.sample_estimator(model, 10, feature_list, train_loader)

outf = './output/resnet_cifar10/'

/home/zjiaming/anaconda3/envs/randomTexture/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


/home/zjiaming/deep_Mahalanobis_detector/lib_generation.py:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data = Variable(data, volatile=True)



 Training Accuracy:(99.75%)



In [2]:
class AC(torch.nn.Module):
    def __init__(self, model, num_classes, outf, out_flag, net_type, sample_mean, precision, layer_index, magnitude):
        super(AC, self).__init__()

        self.model = model
        self.layer_index = layer_index
        self.sample_mean = sample_mean
        self.precision = precision
        self.num_classes = num_classes
        self.layers = layer_index

        self.linear = torch.nn.Linear(5, 1).cuda()

    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    def forward(self, x):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        confidence_score = 0
        for layers in range(self.layers):
            out_features = self.model.intermediate_forward(x, layers)
            out_features = out_features.view(out_features.size(0), out_features.size(1), -1)
            out_features = torch.mean(out_features, 2)

            # compute Mahalanobis score
            gaussian_score = 0
            for i in range(self.num_classes):
                batch_sample_mean = self.sample_mean[layers][i]
                # zero_f = out_features.data - batch_sample_mean
                zero_f = out_features - batch_sample_mean
                term_gau = -0.5*torch.mm(torch.mm(zero_f, self.precision[layers]), zero_f.t()).diag()
                if i == 0:
                    gaussian_score = term_gau.view(-1,1)
                else:
                    gaussian_score = torch.cat((gaussian_score, term_gau.view(-1,1)), 1)
            
            gaussian_score, _ = torch.torch.max(gaussian_score, dim=1)
            
            gaussian_score = torch.unsqueeze(gaussian_score, dim=1)
            if layers == 0:
                confidence_score = gaussian_score
            else:
                confidence_score = torch.cat((confidence_score, gaussian_score), 1)

        outputs = torch.sigmoid(self.linear(confidence_score))
        return outputs


In [3]:
d_model = AC(model, 10, outf, True, 'resnet', sample_mean, precision, 5, 0.0)


In [4]:
# for param in d_model.parameters():
#     print(param)
std_weight = torch.tensor([0.03915799,  0.02396634, -0.09159853, -0.00617044, -0.00793493], requires_grad=True)
std_bias = torch.tensor([-11.964], requires_grad=True)
with torch.no_grad():
    print(d_model.linear.weight)
    # print(model.linear.bias)
    d_model.linear.weight[0] = std_weight
    d_model.linear.bias[0] = std_bias
    # print(model.linear.bias)
    # print(model.linear.weight[0])
    # model.weight = torch.nn.Parameter(torch.ones_like(model.weight))

Parameter containing:
tensor([[-0.3531,  0.1318, -0.0377,  0.0889, -0.2183]], device='cuda:0',
       requires_grad=True)


In [6]:
d_model.eval()

model.cuda()
d_model.cuda()
from attacks import ood_pgd
attack = ood_pgd
ce_loss = torch.nn.BCELoss()
soft = torch.nn.Softmax(dim=1)

model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, requires_grad = True), Variable(target)

        d_target = torch.unsqueeze(torch.ones(16), dim=1).cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        # print(ce_loss(d_model(data), d_target))
        print("before", d_model(data))
        print("before confidence", soft(outputs.data))
        # print(outputs)
        # print(d_model(data))
        with torch.enable_grad():
            attack_inputs = attack.perturb(model, data, data.clone(), target, num_steps=10, attack=True, step_size=0.07, epsilon=0.031, celoss=True)
        print("after attack", soft(model(attack_inputs).data))
        with torch.enable_grad():
            inputs = attack.perturb(d_model, attack_inputs, attack_inputs.clone(), d_target, num_steps=20, attack=True, step_size=0.07, epsilon=0.031, celoss=False)
        # with torch.enable_grad():
        #     inputs = attack.perturb(model, data, data.clone(), target, num_steps=1, attack=True, epsilon=0.0031)
        print("after", d_model(inputs))
        print("reduce ood", soft(model(inputs).data))
        break
    


before tensor([[0.1829],
        [0.0234],
        [0.0399],
        [0.4384],
        [0.0616],
        [0.0122],
        [0.2953],
        [0.0062],
        [0.0017],
        [0.0009],
        [0.0032],
        [0.0056],
        [0.0144],
        [0.0077],
        [0.0026],
        [0.4761]], device='cuda:0')
before confidence tensor([[3.0283e-05, 4.2229e-05, 1.0920e-05, 9.9961e-01, 1.4067e-05, 1.3729e-04,
         1.2954e-04, 1.1758e-05, 7.7413e-06, 5.9452e-06],
        [2.0396e-05, 6.4483e-05, 5.0365e-06, 9.4132e-06, 4.3504e-06, 6.7227e-06,
         1.1908e-05, 5.6047e-06, 9.9987e-01, 6.0123e-06],
        [1.4476e-03, 7.5674e-01, 6.0230e-04, 3.8191e-04, 4.5819e-04, 5.7109e-04,
         6.9231e-04, 5.3860e-04, 2.3446e-01, 4.1062e-03],
        [9.8827e-01, 3.2023e-04, 5.1467e-03, 5.0750e-04, 1.5264e-04, 2.0724e-04,
         3.4502e-04, 3.3499e-04, 5.4772e-04, 4.1680e-03],
        [2.4646e-05, 7.7813e-05, 1.8792e-04, 6.1568e-05, 7.3602e-05, 2.7898e-05,
         9.9947e-01, 1.7561e-05,

In [7]:
# i = 0
# for param in d.parameters():
#     if i > 109:
#         break
#     param.requires_grad = False
#     # print(param, i)
#     i+=1
in_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

train_loader, test_loader = data_loader.getTargetDataSet('cifar10', 16, in_transform, 'output/resnet_cifar10/')


Files already downloaded and verified
Files already downloaded and verified


In [142]:
d_model = AC(model, 10, outf, True, 'resnet', sample_mean, precision, 5, 0.0)

learning_rate = 0.001
criterion = torch.nn.BCELoss()
# criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

out_train_loader, out_test_loader = data_loader.getTargetDataSet('svhn', 16, in_transform, 'output/resnet_cifar10/')

for epoch in range(1):
    i = 0
    for data, data2 in tqdm(zip(train_loader, out_train_loader)):
        data, labels = data[0].cuda(), data[1].cuda()
        data, labels = Variable(data, requires_grad = True), Variable(labels)
        labels = torch.ones(labels.size()[0]).cuda()

        out_data, out_labels = data2[0].cuda(), data2[1].cuda()
        out_data, out_labels = Variable(out_data, requires_grad = True), Variable(out_labels)
        out_labels = torch.zeros(out_labels.size()[0]).cuda()
        # print(data)
        data = torch.concat((data, out_data), 0)
        labels = torch.concat((labels,out_labels), 0)
        optimizer.zero_grad()
        outputs = d_model(data)
        outputs = torch.squeeze(outputs)
        # outputs = outputs.view(outputs.size(0))
        # print(outputs.shape)
        loss = criterion(outputs, labels.type(torch.float))
        # print(loss)
        # break
        # loss = Variable(loss, requires_grad=True)
        loss.backward()
        
        optimizer.step()
        i += 1
        if i % 100 == 0:
            print(loss.item())
    


Using downloaded and verified file: output/resnet_cifar10/svhn-data/train_32x32.mat
Using downloaded and verified file: output/resnet_cifar10/svhn-data/test_32x32.mat


105it [00:04, 23.30it/s]

33.490150451660156


204it [00:08, 23.26it/s]

32.75642776489258


...

3003it [02:09, 23.26it/s]

33.40641403198242


3102it [02:13, 23.00it/s]

36.5000114440918


3125it [02:14, 23.20it/s]


In [8]:
M_in = get_Mahalanobis_score(model, test_loader, 10, outf, \
                True, 'resnet', sample_mean, precision, 0, 0.0)

NameError: name 'get_Mahalanobis_score' is not defined

In [ ]:
def get_Mahalanobis_score(model, test_loader, num_classes, outf, out_flag, net_type, sample_mean, precision, layer_index, magnitude):
    '''
    Compute the proposed Mahalanobis confidence score on input dataset
    return: Mahalanobis score from layer_index
    '''
    model.eval()
    Mahalanobis = torch.tensor([1]).cuda()
    # Mahalanobis = []
    
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, requires_grad = True), Variable(target)
        
        out_features = model.intermediate_forward(data, layer_index)
        
        out_features = out_features.view(out_features.size(0), out_features.size(1), -1)
        out_features = torch.mean(out_features, 2)

        # compute Mahalanobis score
        gaussian_score = 0
        for i in range(num_classes):
            batch_sample_mean = sample_mean[layer_index][i]
            # zero_f = out_features.data - batch_sample_mean
            zero_f = out_features - batch_sample_mean
            term_gau = -0.5*torch.mm(torch.mm(zero_f, precision[layer_index]), zero_f.t()).diag()
            if i == 0:
                gaussian_score = term_gau.view(-1,1)
            else:
                gaussian_score = torch.cat((gaussian_score, term_gau.view(-1,1)), 1)
        
        gaussian_score, _ = torch.max(gaussian_score, dim=1)
        Mahalanobis = torch.cat((Mahalanobis, gaussian_score))

        
        # # Input_processing
        # sample_pred = gaussian_score.max(1)[1]
        # batch_sample_mean = sample_mean[layer_index].index_select(0, sample_pred)
        # zero_f = out_features - Variable(batch_sample_mean)
        # pure_gau = -0.5*torch.mm(torch.mm(zero_f, Variable(precision[layer_index])), zero_f.t()).diag()
        # loss = torch.mean(-pure_gau)
        # loss.backward()
         
        # gradient =  torch.ge(data.grad.data, 0)
        # gradient = (gradient.float() - 0.5) * 2
        # if net_type == 'densenet':
        #     gradient.index_copy_(1, torch.LongTensor([0]).cuda(), gradient.index_select(1, torch.LongTensor([0]).cuda()) / (63.0/255.0))
        #     gradient.index_copy_(1, torch.LongTensor([1]).cuda(), gradient.index_select(1, torch.LongTensor([1]).cuda()) / (62.1/255.0))
        #     gradient.index_copy_(1, torch.LongTensor([2]).cuda(), gradient.index_select(1, torch.LongTensor([2]).cuda()) / (66.7/255.0))
        # elif net_type == 'resnet':
        #     gradient.index_copy_(1, torch.LongTensor([0]).cuda(), gradient.index_select(1, torch.LongTensor([0]).cuda()) / (0.2023))
        #     gradient.index_copy_(1, torch.LongTensor([1]).cuda(), gradient.index_select(1, torch.LongTensor([1]).cuda()) / (0.1994))
        #     gradient.index_copy_(1, torch.LongTensor([2]).cuda(), gradient.index_select(1, torch.LongTensor([2]).cuda()) / (0.2010))
        # tempInputs = torch.add(data.data, -magnitude, gradient)
 
        # noise_out_features = model.intermediate_forward(Variable(tempInputs, volatile=True), layer_index)
        # noise_out_features = noise_out_features.view(noise_out_features.size(0), noise_out_features.size(1), -1)
        # noise_out_features = torch.mean(noise_out_features, 2)
        # noise_gaussian_score = 0
        # for i in range(num_classes):
        #     batch_sample_mean = sample_mean[layer_index][i]
        #     zero_f = noise_out_features.data - batch_sample_mean
        #     term_gau = -0.5*torch.mm(torch.mm(zero_f, precision[layer_index]), zero_f.t()).diag()
        #     if i == 0:
        #         noise_gaussian_score = term_gau.view(-1,1)
        #     else:
        #         noise_gaussian_score = torch.cat((noise_gaussian_score, term_gau.view(-1,1)), 1)      

        # noise_gaussian_score, _ = torch.max(noise_gaussian_score, dim=1)
        # # print(noise_gaussian_score)
        # Mahalanobis = torch.cat((Mahalanobis, noise_gaussian_score))
        # Mahalanobis.extend(noise_gaussian_score.cpu().numpy())

    return Mahalanobis

In [54]:
# torch.empty((10000))[0] = torch.tensor(5)
torch.gradient(M_in[1:])

(tensor([17.2188,  9.3410,  1.6312,  ..., -1.2758,  1.6002,  1.8951],
        device='cuda:0', grad_fn=<CatBackward0>),)

In [51]:
M_in = get_Mahalanobis_score(model, test_loader, 10, outf, \
                            True, 'resnet', sample_mean, precision, 0, 0.0)

In [12]:
M_in    
# len(M_in)
# model = LogisticRegression(5, 1)
# torch.squeeze(model(X_train)).round().detach().numpy()

tensor([[ -7.8309,  -7.8422,  -4.4515,  -4.7944,  -3.8910,  -4.7397,  -3.7094,
          -4.7983,  -8.9876,  -7.9356],
        [-14.0703, -11.5042, -10.5742,  -9.2915, -11.0764,  -9.4256, -10.4809,
          -8.4230, -15.2782, -10.7860],
        [ -7.8402, -12.6759, -11.3069, -11.6400, -12.1074, -12.2116, -12.6828,
         -12.3974,  -9.7964, -13.1036],
        [-11.1188, -13.9470, -13.5376, -15.1099, -14.2422, -15.1046, -15.3131,
         -15.6443,  -9.2849, -14.3039],
        [ -8.7679,  -8.5532, -13.2916, -14.3393, -14.5887, -15.8007, -14.4166,
         -13.8077,  -7.5496,  -8.6151],
        [-21.7121, -22.3444, -15.8825, -18.6516, -15.0568, -18.6731, -15.4293,
         -17.9079, -23.3436, -22.4998],
        [ -9.8352,  -7.9063,  -7.6867,  -6.3545,  -8.2395,  -6.1512,  -7.7033,
          -5.9360, -10.9791,  -7.1236],
        [ -9.3610, -10.7226, -13.5705, -13.5829, -14.3001, -14.5927, -14.5812,
         -13.4367,  -9.2280, -10.2005],
        [-17.8524, -17.5579, -17.2364, -14.6830,

In [62]:
Y_train

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 